In [ ]:
!pip3 install shapely==1.7.1

In [ ]:
!pip install google-cloud-aiplatform
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 5.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.4 MB/s eta 0:00:00


In [ ]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/ucr-ursa-major-maverics-lab-5986748a3acf.json"


In [ ]:
import pandas as pd
dftest=pd.read_csv("/content/extractive_test.csv")
dftrain=pd.read_csv("/content/extractive_train.csv")

In [ ]:
dftrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4352 entries, 0 to 4351
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Article ID         4352 non-null   object 
 1   Original Article   4352 non-null   object 
 2   Highlight          4352 non-null   object 
 3   Generated Summary  4352 non-null   object 
 4   R1-R               4352 non-null   float64
 5   R1-P               4352 non-null   float64
 6   R1-F               4352 non-null   float64
 7   R2-R               4352 non-null   float64
 8   R2-P               4352 non-null   float64
 9   R2-F               4352 non-null   float64
 10  RL-R               4352 non-null   float64
 11  RL-P               4352 non-null   float64
 12  RL-F               4352 non-null   float64
dtypes: float64(9), object(4)
memory usage: 442.1+ KB


In [ ]:
dfllm= pd.read_csv("/content/llm_with_roguescores.csv")

In [ ]:
P=['0408464fb6eb2702626d2ca1cbb7c28e76ddd255', '18f7505fa80eaa981a2f52fbb96fcea5908cd4ee', '97163bb36cb49730a634a0cbaa01108c2fd89e01', 'cd421645a56af17128c17a41ec2c9074bce84ddc', '4fde6cc846d0709e25a9cccec8d197ba26c15c82', '5f5a8814d8ec3c16299782db7c724cb771ea5adb', '56516cebdf30bd22dc8353549b70b4bb33993753', '446fdf91f59be259167ae59b0a3de1a664891751', '3d58a02a6578e08c2bf7494263ef9a083620e1f6', 'c077e15f710484d2cb3ffe26cd9b331353cb74e2', 'fd7a007f17a13ee329cba78b298bdd77d4d33d42', 'ba2ec653b7d0c46759f39aef6dac850c9229716f', '4cfe02fe65eedf9364dd2b9633dca27349ec02aa', '58b0d34958a722a453fe24deed240e08e7be8ced', '6abc848961744dde2da8a4b8e5337f34d32d45ed', '62545d81f24bc768afd19875d9e62e38a6ddb97d', 'efdb6043cf2ac2663137f8cb8696f0735ee58b86', 'bde8e22f3e1c3af2ca0f09f734f33d67140ef1aa', '9eccb748cdaefd4c472101db76f347914f850c1a', '685b0b902225d5b807a721e294f5ae16febf1f0e', 'b4c8c6a6ad6257b5d8dfd82b80cd3b0fbdd342e3', 'e79e40746dbc24643a3a3f044d2f49689ff82e38', '539d895fd424cbda3e4985c6aee36e272401b359', 'db61d01f82dc63596f7e2fa93b4ab4ae278daf1f', 'c331159c09eee2e7a27b90d973500e7b3db04d94', '1f4aaf00cad1c6272240bd109fd34201b92a0937', '436772093dd75272d1d867c80e6b142329fcfa3e', '6de323acb06b2590f5aba38ae450865ea4e793b1', 'ebccdbf5779c2d3fa7180747e333dde756aa62fb', '2bc2c4566f364d664331809744ff9a40841d9617', '975132a0649fbebbdb6cdd32534b165eece0053a', '3612c620d3fa1eafc99c0c154ad5caf7481727f4', '31edccbcedf3d7e2819818f109a35ecdfcd65970', 'b8d8f6a6518138c8b1e0e845f46e62e2b7fe3fd8', 'de45d9d92d7ee486a71ba13f4cc52470b9df5c01', 'be8918aac4ce45cfefddaa245c26d9314d1194d4', '3415b0e398cd14d27f386b28a61f4d585b8eaa11', '7623d447bab49c8e99dfc99289b2281a1fed8d4f', '45071cfaf26fa14f1a8a9276393401f54e0f93ca', '837970f6b544e927c6fe2261a81f74f8313f2bdf', '47609113b2709b46ccafb3782980cd3da7ab7d2d', 'c446f86eea5f7d48f5cae27df669b80aff8de043', 'a84811a80358e86f9a62669fa69521a0ac12130a', 'bbdc0e58a203b016166f896936d9862ae9fd11be', '6c5ef5f577b97c98b8580476690883176584569e', '6a778af88ba885ef38fd88987afa1b6167e5c494', '2094d594b67b61d842fdc5ef4891b721e36171e7', '6af327c6e93d12422c1b605ff7581a813cdb1691', 'c94b92e2e7f31e2ed580137a02a5c60fa6537458', '540fc636957b4d049d6b4f07ae72c1b3be6ec020', 'c8fcf4c23a15895a2c7272ec04c5a92b06575d5e', '3a407cf2ec50f113c6300b7a39936c229d1b29f4', '1d17d402757c7da68c499b95368f477ad5d66513', 'a382fc98c0652dea0c2275a4110b744476b30ea3', '949378fdf5114619911a5e0fe0182d45f1c27c17', '9dad1056763b4580aed22d85670a801c8c3f352c', '644da4a1c3baef155cf8c4675008540a8b777988', '35d5a717be7c0144d24c34233e5321e871dc614e', '8000bfa6f05e428b2c7cc552d0df4c36283108fd', '7be8bc293e448903d2b23f4cea091b0407848897', 'bfcee0d2eed31196bb00e1f52606a3f168eee17c', '2b4eb5a283f51b54e1b0318936ae9bcfa70ce243', 'd75247e02a692fc343bc9826662870011eaed521', '4b637c62b6ac747871a3051a97f29dd282633360', '270fdae145aa5ae3ae73adcf44be72cec339dbbe', '60c3114cd31fced2982a5e912f2224220d6c5254', '32259d1946a7e376f3915aea7f7b4a4e14cbe02f', '80a28afe6ded1ad0ddd6f2ba0ac9b66436e91739', 'c3881866bcacbb55bd959b15a197bae3eb8b3384', 'f5ff04170c39e33afdf58bb732dd03265662c4c0', 'cb7a72d78d5f34b9300f8486e40e57ffcf9d98d5', 'f1beee2435f59b05e0da484c7a1c167709412d6d', 'e1502ae8cb04cbdceee368551cfd65bcf6a16f07', 'e6e0a4b568f6214c18c38cfcac5a10ecfb0dbf6e', '623ad83ebb32752afaa1fb4b134ab697eae8b798', '0e6dd3f40daaff5c14f0a34ba75e9f05c40a55c8', '0d1d6284061a99cb2478ecb4b490e4605698ec3e', 'de24115848174d0534512cd251b1d53cd343f23e', '574767c29dfee14ea227639f45387c07437a55e5', 'a8a2ace82e62b4020b5a241060d4a548f7ee3a7f', 'bef9d7ac62dac0fb9a200d2acae6d333388ec7f1', '6426eaaa8fe11407ed2290d9c0ad2bc8e1a723d5', 'bb8a7a6bc30f5d10bfd747daa4887134a780d66d', 'f7dd3331408529408be2b706392e0b4a02a6a4cc', '84a933b7edc70c7f5e6ed6558c7b57c100abc114', '9f4ab634ce7925b5527345ba0efc583b206c47a2', 'f800996f2cca42b205a7cfd37805fb77d577254b', 'd6f928d950f7d97c9fc3a5f575b9eaa325a7dd4d', '8f324e3064f6cdc9c378dfcf4545ed2c124e7103', '32069f98ff75ce1a65beeffcf84cd80f92bae5dc', '7f13d9dff7707609a042a5c3e99671c98de8c135', '4dcf206f63daa3ffdb4c77b2441e7aed78d2c8c5', '864240ad271c7da6c74823abf2abfb2b98f427cb', '6299e17b582fffbd6732d237dcc06c97c5d935ff', 'c3cb61c858ec4a178760b619430ddff7e03eb0c8', '0a8fd2888f20c9e705696d8ef847a32df47900e6', 'b68be4e8fe382de849954933357e4a32409c20ff', '34f1b80e4e3fbd60dd942b3e7e52a7f8c61f06ef', '52343da9a3ef719280b4b6946e6c500e819552d0', '2f6106809de41d59f422d527900a6b344174c5b2', '69406e34836ea1f7680d4298e83fd6d83a6c7997', 'ef2d42928cc3b1762d0bbb6037db308db53fc864', '31809d739569d22b663fc1cdbfb1fb4206ade734', '98d980ba9bc5479f9c340a18a61941d5ccb52774', 'dbba46c94632ad510072e1b52c4f427c4dadca60', '65676acfb425e2a29cd7fca216174635ea408382', '16b9db5d22a87f739c918886c6b3d7acdfcfd58a', '30f2056f43c0a58a261c4feddefd131d498199cb', '5231a93ff48e3dcdc22ea9df981237c26a474714', '29503dc74ccb039c4c1562844299116a545dea39', 'd34233997a6d71174a09a8d8001ec64468a2f640', 'a37aa9e7b6985f710c3ad79cd0b53619b5e0bf5a', '707c09009396d151d2d4f819925f78e446151f39', '33ca9fba99197c82369e99aa3d234783e719227c', 'faf1d451489b43676734ea706926885bbbedfce3', '0c5ec67f7b1d7a2fdb0a3f3ffb8263d8d9ddf872', 'd9e8e6f9c0255bf26881a2eabba8f5cbe733c55f', '9625484feb731f45fd8059112238d94f141d413c', 'ca3b2dc93f170f06e3a60fe7cd3243d204cc4b75', '2103c883437dbd9f0c17885f60190dabb4ab56dc', '30944082061b9e844391dcbc98775f22cb3ef791', '202c35ec48be21fc196139dd538db271c0f65e4a', '280d5930fb82ad2958fd1dcf452c6a4f1b5ad39f', '71efc54d9a237c40c469965cc24841f9f9a7b019', '3291399360e20650ea565c27371a72c7b448082e', 'e67604ecb75aa21391e1d15a577d16b8db990de5', 'e2a44a3e3edd44c394ad6ff7207980f7f1e9b4b3', '7661803c32a1511bf302a8a5abdc3c462f080fb2', 'de84417cc1ffcb6ecabfab7600c232eb9fd11915', '8a7466a5c8516b2087aea24647cc07b508697ab6', '20a6b5894c4cd75e32f6f4946467854aeb539337', '7113eca13e8f3dd7760e609fdc97660de1806d2a', '61254109741e1db17ba31582d1c783d282e6bbf6', '78b863c5bce9b8fc0d743bc396b0308a9eb17a77', '92151f31c3478c5e0e4352a0f7323c3acfc297e7', 'cabca761a3814275d3d1b2b6c7cfc1f9cd3810b7', '878c4af5ebfb7a1831af90ad0d6e7bdcc4a3d4b8', '1dd00c89d71a5611797bb34da8a6eada8f058405', '711e00313310a956641a47791fdf8299ee06b2c2', '01b0585316ca58505df1d4ace2d888d0e50e7600', 'c11e656ad31a4f14b0d276290a757a0da6a73ba4', '4b132f90a0f4369df9b7ecba759c5aa068dd12be', 'd3e9f0aca4e608fc2a3b85115b96cb0c453191af', 'c18fe1d648b40aa763644c37a9a2a562a340e172', '9ad4662fc2d978672ae4b5eea4e4806ae6a7d076', 'db9f358d0e08da7cd0276453f8bcdc114ff06fdb', 'b19ab7fa421bcbb921c5a20718db5918091bb15d', '9268b9d826db3d8acf31a03bdc3e30cd75cc93de', 'be5d8524bf21d485e203425d764ed60a3bf71b8d', 'f97b9b19e5a90dce0fc0b098f2a7f70ec36445fb', 'e84c74b45a589a6faefaa881016505fd282a46ca', 'd495bb584d5fda76f9b90beca528469f5bcfc5b3', 'f7f9156f26bbf12e63523bdd2f607a2f7a7f9f0c', '7e91926e47bdaa4054e3a69784bee8ee27d0dd4b', '2434b0ec4419fd48c0d15e543030555129398ec6', 'f0dba7fa5d45ffb29070afda0c4b230c8f318a51', '82461522ee90f8f5be2ec226e2016ad40588866d', 'a3830cf0f4311a3f816a710d61d7e14ac9ffa9fe', 'bc9d5ab2c343edd6522ee4ebe63c5ca4adcebb1b', '9638b449931f0f4a21167997d37d5dd43ee2430f', 'e8b93366b9c9c5bf4409dc1f8064122063ccd4ca', '3a8e2afdba04d7356526c94232f17eef9a5e4ddd', '7ae16f8ac4a86a6de4669e130e068d9f7e492ddf', '1b3bdc7fa131c4c82e8e996c936abe0da840e2ef', '5d1577700e760d853376d15ad75d8972119d6ddb', 'd6bbbfa9b5c58e99d21d549a9fb5b512b342e301', 'e0a415bf04e1929507bb8626cdf6379169264838', '5c3e16c415b463269527cafd220170e1303048f8', '09085a0b380bcbc205c7a3902f4d5492da2eacda', '223ecfeb056056991eb877fbd63ce66ca616d372', 'f708809379316d68524802c6ca052985bbe248cd', 'c38207a8ba7332bf8fc449770b2554510fa1482a', '3bdec3398b4dd68453d97ab7718086655843e899', 'a7827e12988bd89e4f1079705079a135beefc1f2', '34c33427589efd00f343e49e2e5109d444d560f5', '198f9a4a72b169e04ec003cb4717f46dc0d3191f', '9ef9bcf7b102537cff2774dd7be2c6eefd5b80fd', '7e4d197d5fe8272c0471199cf9987f3786a80c2b', '1fda6d5647b826db2ab920c945dd7944dbdfbde9', '2805d03b2ec2d8a1e5343f71d70912db2a58736e', 'ddcbe22893aa887ff799cf3e40bbe2a536965f91', 'b6d1933d8d4a8c1b6b76d44ec3ed69d86268a7b3', 'ad58f533f9eaf52080929e32c5dd5eaf2bafba0e', 'f2a9f3c80876424da8cee335cb0e0467caa5f3ca', '5ba4a7a9733a497625971981764184cb75618b81', 'a659d867e198efa85e02b90d5e78d2f6a6e60fb7', '36c34f4ffa8815711beea7d2fda86eea047b8802', 'a4b00d5247eef69b5d51cab1f1883da314211366', 'cca91bf7af4db82293421776f5299457dce34909', 'e248a8e2416dddfa8899f0781b967429232063b3', 'e266b3daac6d9e72e637e8f83db97621c5beafd0', '9f2dbe8953dd5b1d2b0f2f748c51bfccbe72906e', 'a2b66e4376258cd41604d654a395f7540d2e8514', '2edc424a6d80e2e10a46b1e8b52f14c481cc1daa', '0cdf69eb6b6860142724e940cf9f75d93907227f', '34ad340fa41e205b95b37b00e8ccbc38afb3e68f', '117444537cbffcb807dc04565f39dca46f7d2cef', 'ecb7f439cc992286b8933b117bb6c498bd133bcd', 'b6ee1b3428dda9063461deefc0086c2d66279d0f', '42d88bafeb5f838a0be02c3eb97fe12f5df3d92f', '4c976ed4cb92068d476b1c17753715948e39c62f', 'a5fffe339cda4c8e002d992ab93888559107f3ab', 'd6fc70d27119a3705b96ace6b20b9fd2f3cfcbca', '89fed7685538cd358e611932653eea931d7b057f', '5effc218f677590dbc9037e196549b5341242933', 'b8a47e9569e9d0919638f5bed6f13e565f84e855', '35c590581dcd2a417101f2ef80f63f69cdfe4030', 'f99b61fb576e893ffbd2e6d6f494b9dd0fbf8941', '5dd71e881a79e27363e2f73c3afa80b8cc3e5a64', '64b3b703816b40851bd4b7c0255df5834e136611', 'cf5e6722047569c81b635be96ef0b07cb04eed73', 'ffd4fd3ed5e3d498d6135fc93ae6c9170d10611c', '159ab1d0f729f77e86a6a77bf43d0291d069c66c', 'e0a3acedc34d92569ce5f6032d504c35fbf30720', '0e5130d172cdaaa2afcb51585f4aa0580b049431', 'a776fa2caa0fa9cac0ce5bf71d2bf44eb7a7346a', '1027d076dbb423b90447f6f3f8315610aefd682c', '0322c7a3366d45134e8ba78c7f369d2f2cd14176', '0db1a9abd0524d51270531e3a80d36afaa7550fe', '0d29ae06dd015e141646c3b03671901c1b8465c4', 'e2ef75518e69947f79acb9202516689b6c869e92', '16bf321c82f6ffe7f20479a12a9e432685f2d91b', 'c37431a19d8433f20447858d1c810b7b77804473', '77685c1f281a3e7472b51f6619d7d54c6bc0c717', '6d76062fb07e8a37609bbd527934a9219929560b', '083b6c3239f969ae12491f14fc09b1f17d396983', '1afd2df29d4e642f20130a231d27004216667fe1', '8749719d4fb9a14d833d39cde0b6e07985bee823', '43959f19ee64571fead0661415d4e13c54d71ee1']


In [ ]:
len(P)

229

In [ ]:
dfllm.head()

,Article ID,LLM Summary,llmR1-R,llmR1-P,llmR1-F,llmR2-R,llmR2-P,llmR2-F,llmRL-R,llmRL-P,llmRL-F
0,517a2943ee6fab9e6a81bd5cd259e5801682d190,Jack Grealish is a 19-year-old academy graduat...,0.172414,0.078125,0.107527,0.000000,0.000000,0.000000,0.137931,0.062500,0.086022
1,e66247c8e4f0f7907e9c040bf7064782dfcb3323,"Memories Gourmet Pizza in Nichols, Wisconsin w...",0.200000,0.166667,0.181818,0.050000,0.041667,0.045455,0.171429,0.142857,0.155844
2,907fe50dbe4dc0fb84b0683fab578806536dba28,Shay Given will start for Aston Villa against ...,0.527778,0.395833,0.452381,0.225000,0.163636,0.189474,0.527778,0.395833,0.452381
3,41a572feb1c9f4651b9b1e78f1482b92e16a63b1,Amir Khan believes Kell Brook has called him o...,0.317460,0.170940,0.222222,0.047619,0.021164,0.029304,0.317460,0.170940,0.222222
4,bb4019681e575f38978b27b51ccdc3ee07946e2c,Andros Townsend scored a superb equaliser for ...,0.444444,0.149533,0.223776,0.139535,0.035928,0.057143,0.444444,0.149533,0.223776


In [ ]:
template = """Write a concise summary for the following: "{a}." """
prompt = PromptTemplate(template=template, input_variables=["a"])

def get_summary(a, retries):
    llm = VertexAI(
        project='ucr-ursa-major-maverics-lab',
        model_name="text-bison@001",
        temperature=0.2,
        max_output_tokens=1024,
        top_p=0.8,
        top_k=40
    )
    if retries < 0:
        return None
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    b = llm_chain.run(a)
    if 'article' in b or b == '' or b is None:
        return get_summary(a, retries-1)
    else:
        return b


def generate_summary_for_one(article_id, article_content):
    summary = get_summary(article_content, 10)
    if summary is None:
        return None
    return summary

def generate_csv_for_articles(df):
    results_df = pd.DataFrame(columns=['Article ID', 'LLM Summary'])
    summaries_generated = 0

    for index, row in df.iterrows():
        if summaries_generated >= 229:
            break
        article_id = row['Article ID']
        article_content = row['Original Article']
        summary = generate_summary_for_one(article_id, article_content)
        if summary is not None:
            results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)
            summaries_generated += 1
            print(f"Generated {summaries_generated} summaries so far.")
    results_df.to_csv('llm_summaries.csv', index=False)

generate_csv_for_articles(dftrain)



<ipython-input-39-078aecbbcd33>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)


Generated 1 summaries so far.


<ipython-input-39-078aecbbcd33>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)


Generated 2 summaries so far.


<ipython-input-39-078aecbbcd33>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)


...

<ipython-input-39-078aecbbcd33>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)


Generated 227 summaries so far.


<ipython-input-39-078aecbbcd33>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)


Generated 228 summaries so far.
Generated 229 summaries so far.


<ipython-input-39-078aecbbcd33>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Article ID': article_id, 'LLM Summary': summary}, ignore_index=True)


In [ ]:
rest=pd.read_csv("/content/rest.csv")

In [ ]:
!pip install rouge

In [ ]:
# Import necessary libraries
import pandas as pd
from rouge import Rouge

# Define the calculate_rouge function
def calculate_rouge(hypothesis, reference):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference, avg=True)

    # Extract scores and format them into the desired dictionary format
    formatted_scores = {
        'llmR1-R': scores['rouge-1']['r'],
        'llmR1-P': scores['rouge-1']['p'],
        'llmR1-F': scores['rouge-1']['f'],
        'llmR2-R': scores['rouge-2']['r'],
        'llmR2-P': scores['rouge-2']['p'],
        'llmR2-F': scores['rouge-2']['f'],
        'llmRL-R': scores['rouge-l']['r'],
        'llmRL-P': scores['rouge-l']['p'],
        'llmRL-F': scores['rouge-l']['f']
    }
    return formatted_scores

# Read the llm_summaries.csv file into a DataFrame
results_df = pd.read_csv("rest.csv")

# Assuming df contains the reference summaries and is already loaded
# Calculate ROUGE scores and split them into separate columns
rouge_scores = results_df.apply(lambda row: calculate_rouge(row['LLM Summary'], dftrain[dftrain['Article ID'] == row['Article ID']]['Highlight'].values[0]), axis=1, result_type='expand')

# Concatenate the original DataFrame with the new ROUGE score columns
results_df = pd.concat([results_df, rouge_scores], axis=1)

# Set the display options to show the full content
pd.set_option('display.max_columns', None)

# Print the DataFrame with the new ROUGE score columns
results_df


,Article ID,LLM Summary,llmR1-R,llmR1-P,llmR1-F,llmR2-R,llmR2-P,llmR2-F,llmRL-R,llmRL-P,llmRL-F
0,f001ec5c4704938247d27a44948eebb37ae98d01,The Palestinian Authority has officially becom...,0.451613,0.237288,0.311111,0.235294,0.108108,0.148148,0.419355,0.220339,0.288889
1,88e7dac40f3e6c159e4edc0747d0cc0f50886fbb,"Robert H. Schuller, the founder of the Crystal...",0.535714,0.114504,0.188679,0.218750,0.035000,0.060345,0.535714,0.114504,0.188679
2,4495ba8f3a340d97a9df1476f8a35502bcce1f69,Mohammad Javad Zarif is the Iranian Foreign Mi...,0.551724,0.134454,0.216216,0.281250,0.049724,0.084507,0.517241,0.126050,0.202703
3,2bd8ada1de6a7b02f59430cc82045eb8d29cf033,"Bob Barker returned to host ""The Price is Righ...",0.300000,0.171429,0.218182,0.090909,0.052632,0.066667,0.300000,0.171429,0.218182
4,1b2cc634e2bfc6f2595260e7ed9b42f77ecbb0ce,A high school basketball player asked a freshm...,0.478261,0.224490,0.305556,0.173913,0.056338,0.085106,0.478261,0.224490,0.305556
...,...,...,...,...,...,...,...,...,...,...,...
224,788c4a340d6aa99163bb63e2c58ffae12ee4824b,The Food and Drug Administration is considerin...,0.387097,0.200000,0.263736,0.090909,0.038462,0.054054,0.322581,0.166667,0.219780
225,daa7e67d12ba72cd573f0e484e8980f884f00177,America has the highest incarceration rate in ...,0.296296,0.216216,0.250000,0.241379,0.170732,0.200000,0.296296,0.216216,0.250000
226,ae4c185040457dae1d6a9fe4b9f9cc8a3997bcc6,2 people from different generations and politi...,0.263158,0.169492,0.206186,0.051282,0.028571,0.036697,0.236842,0.152542,0.185567
227,ccb37f6eba3568f515734a47646183b3fb27c998,cnn reports on a skywest airlines flight that ...,0.421053,0.235294,0.301887,0.000000,0.000000,0.000000,0.368421,0.205882,0.264151


In [ ]:
results_df.to_csv('rest_with_rouge.csv', index=False)

In [ ]:
restr=pd.read_csv("/content/rest_with_rouge.csv")

In [ ]:
restr

,Article ID,LLM Summary,llmR1-R,llmR1-P,llmR1-F,llmR2-R,llmR2-P,llmR2-F,llmRL-R,llmRL-P,llmRL-F
0,f001ec5c4704938247d27a44948eebb37ae98d01,The Palestinian Authority has officially becom...,0.451613,0.237288,0.311111,0.235294,0.108108,0.148148,0.419355,0.220339,0.288889
1,88e7dac40f3e6c159e4edc0747d0cc0f50886fbb,"Robert H. Schuller, the founder of the Crystal...",0.535714,0.114504,0.188679,0.218750,0.035000,0.060345,0.535714,0.114504,0.188679
2,4495ba8f3a340d97a9df1476f8a35502bcce1f69,Mohammad Javad Zarif is the Iranian Foreign Mi...,0.551724,0.134454,0.216216,0.281250,0.049724,0.084507,0.517241,0.126050,0.202703
3,2bd8ada1de6a7b02f59430cc82045eb8d29cf033,"Bob Barker returned to host ""The Price is Righ...",0.300000,0.171429,0.218182,0.090909,0.052632,0.066667,0.300000,0.171429,0.218182
4,1b2cc634e2bfc6f2595260e7ed9b42f77ecbb0ce,A high school basketball player asked a freshm...,0.478261,0.224490,0.305556,0.173913,0.056338,0.085106,0.478261,0.224490,0.305556
...,...,...,...,...,...,...,...,...,...,...,...
224,788c4a340d6aa99163bb63e2c58ffae12ee4824b,The Food and Drug Administration is considerin...,0.387097,0.200000,0.263736,0.090909,0.038462,0.054054,0.322581,0.166667,0.219780
225,daa7e67d12ba72cd573f0e484e8980f884f00177,America has the highest incarceration rate in ...,0.296296,0.216216,0.250000,0.241379,0.170732,0.200000,0.296296,0.216216,0.250000
226,ae4c185040457dae1d6a9fe4b9f9cc8a3997bcc6,2 people from different generations and politi...,0.263158,0.169492,0.206186,0.051282,0.028571,0.036697,0.236842,0.152542,0.185567
227,ccb37f6eba3568f515734a47646183b3fb27c998,cnn reports on a skywest airlines flight that ...,0.421053,0.235294,0.301887,0.000000,0.000000,0.000000,0.368421,0.205882,0.264151


In [ ]:
llmsum=pd.read_csv("/content/llm_with_roguescores.csv")

In [ ]:

result = pd.concat([llmsum, restr], axis=0, ignore_index=True)


In [ ]:
result.to_csv('llmsummaries_1000.csv', index=False)

In [ ]:
len(dftest)

1000

In [ ]:
column_list = rest['Article ID'].tolist()

In [ ]:
import pandas as pd

# Say you want to remove rows with Article IDs 'id2' and 'id4'
ids_to_remove = column_list

# Use the isin method with the ~ operator
dftrain = dftrain[~dftrain['Article ID'].isin(ids_to_remove)]

dftrain


,Article ID,Original Article,Highlight,Generated Summary,R1-R,R1-P,R1-F,R2-R,R2-P,R2-F,RL-R,RL-P,RL-F
0,469c6ac05092ca5997728c9dfc19f9ab6b936e40,"marseille , france -lrb- cnn -rrb- the french ...",marseille prosecutor says `` so far no videos...,"marseille , france -lrb- cnn -rrb- the french ...",0.3617,0.3696,0.3656,0.2037,0.1964,0.2000,0.3404,0.3478,0.3440
2,e2706dce6cf26bc61b082438188fdb6e130d9e40,-lrb- cnn -rrb- governments around the world a...,amnesty 's annual death penalty report catalo...,-lrb- cnn -rrb- governments around the world a...,0.4783,0.3333,0.3929,0.2321,0.1548,0.1857,0.4565,0.3182,0.3532
3,c222979bd1cfbc7d3ff821e9c738e3dbd29b14f4,"-lrb- cnn -rrb- on may 28 , 2014 , some 7,000 ...",amnesty international releases its annual rev...,"in the patently unfair , open air trial that f...",0.1765,0.1364,0.1538,0.0000,0.0000,0.0000,0.1765,0.1364,0.1490
4,c27cf1b136cc270023de959e7ab24638021bc43f,-lrb- cnn -rrb- a duke student has admitted to...,student is no longer on duke university campu...,-lrb- cnn -rrb- a duke student has admitted to...,0.7308,0.3958,0.5135,0.2414,0.1186,0.1591,0.6538,0.3542,0.3953
15,c7caf9181c2e94a79d9102da6e11cde48c880aa5,-lrb- cnn -rrb- the united states and its nego...,richard klass : iran framework agreement on n...,-lrb- cnn -rrb- the united states and its nego...,0.3721,0.3265,0.3478,0.0213,0.0179,0.0194,0.2093,0.1837,0.1940
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4347,7e4c604889de6b7da503d8ac2c43108eeb96501f,"until now , it has been a hidden world , prote...",footage reveals huge variety of life on the s...,the icefin was deployed -lrb- and retrieved -r...,0.2903,0.1636,0.2093,0.0571,0.0260,0.0357,0.2581,0.1455,0.1626
4348,d4287f044f2056ef23f53e0ea2a3c54a0ad8ff7a,"arrest : kayla mooney , pictured , allegedly h...","kayla mooney , who is in her first year teach...","kayla mooney , who is in her first year teachi...",0.5854,0.4286,0.4948,0.3830,0.2368,0.2927,0.5366,0.3929,0.4334
4349,9242927b230df6eb617ea4b92b842bd9876f5b61,foreign gps now account for 11 per cent of fam...,foreign gps now account for 1 in 5 family doc...,the number of gps from abroad is up by 11 per ...,0.6111,0.4151,0.4944,0.2564,0.1563,0.1942,0.6111,0.4151,0.4619
4350,114153f5bcef4e19edc4d3c6f6cb313febc939fb,did you blink ? there was a total eclipse of t...,the moon skimmed across the earth 's shadow o...,the so-called ` blood moon ' could be seen by ...,0.3548,0.2157,0.2683,0.1429,0.0714,0.0952,0.3226,0.1961,0.2193


In [ ]:
len(dftrain)

4123

In [ ]:
# Given a DataFrame `df` and a column name 'column_name'
column_list = P


In [ ]:
len(column_list)

229

In [ ]:
new_df = dftest[dftest['Article ID'].isin(column_list)]


In [ ]:
new_df

,Article ID,Original Article,Highlight,Generated Summary,R1-R,R1-P,R1-F,R2-R,R2-P,R2-F,RL-R,RL-P,RL-F
4,0408464fb6eb2702626d2ca1cbb7c28e76ddd255,"-lrb- cnn -rrb- we did it again , in another a...","in baltimore , after the death of freddie gra...","'' you may be saying with some irritation , ``...",0.1000,0.1515,0.1205,0.0000,0.0000,0.0000,0.0800,0.1212,0.0892
5,18f7505fa80eaa981a2f52fbb96fcea5908cd4ee,palermo president maurizio zamparini has slapp...,paulo dybala is being tracked by manchester u...,palermo president maurizio zamparini has slapp...,0.5918,0.5273,0.5577,0.3455,0.2836,0.3115,0.5918,0.5273,0.5540
13,97163bb36cb49730a634a0cbaa01108c2fd89e01,a video has emerged which appears to show a de...,"deanna robinson , of texas , was 38 weeks pre...","deanna robinson , of quinlan , texas , was 38 ...",0.6500,0.7959,0.7156,0.5060,0.6563,0.5714,0.6500,0.7959,0.7015
15,cd421645a56af17128c17a41ec2c9074bce84ddc,newport county have sacked chris zebroski afte...,chris zebroski attacked two taxi drivers in s...,newport county have sacked chris zebroski afte...,0.2045,0.3462,0.2571,0.0566,0.0833,0.0674,0.2045,0.3462,0.2288
27,4fde6cc846d0709e25a9cccec8d197ba26c15c82,-lrb- cnn -rrb- he was impressively polite and...,defense seeks to make case to spare life of c...,dzhokhar `` jahar '' tsarnaev 's defense team ...,0.4063,0.2281,0.2921,0.0500,0.0317,0.0388,0.3125,0.1754,0.1960
...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,6d76062fb07e8a37609bbd527934a9219929560b,a hardware store worker has been found guilty ...,"john r. lind , 34 , from new brighton , minne...","john r. lind , 34 , admitted on thursday to ta...",0.5918,0.5918,0.5918,0.4182,0.4107,0.4144,0.5306,0.5306,0.5306
965,083b6c3239f969ae12491f14fc09b1f17d396983,-lrb- cnn -rrb- authorities in south carolina ...,footage shows a traffic stop and early intera...,-lrb- cnn -rrb- authorities in south carolina ...,0.6000,0.4468,0.5122,0.2093,0.1525,0.1765,0.5143,0.3830,0.4214
970,1afd2df29d4e642f20130a231d27004216667fe1,rome -lrb- cnn -rrb- muslims who were among mi...,"the 12 victims were from nigeria and ghana , ...",italian authorities have arrested 15 people on...,0.5484,0.3091,0.3953,0.2571,0.1216,0.1651,0.5161,0.2909,0.3251
975,8749719d4fb9a14d833d39cde0b6e07985bee823,the tories today launched an extraordinary att...,defence secretary michael fallon launches per...,defence secretary michael fallon claimed the l...,0.5500,0.1692,0.2588,0.2500,0.0625,0.1000,0.5500,0.1692,0.1800


In [ ]:
finaltrain = pd.concat([dftrain,new_df], axis=0, ignore_index=True)


In [ ]:
finaltrain.to_csv('finalexttrain.csv', index=False)

In [ ]:
finaltrain= pd.read_csv("/content/finalexttrain.csv")

In [ ]:
finaltest=pd.read_csv("/content/finalexttest.csv")

In [ ]:
len(finaltrain)

4352

In [ ]:
len(finaltest)

1000

In [ ]:
ext=pd.read_csv("/content/llmsummaries_1000.csv")

In [ ]:
len(ext)

1000

In [ ]:
# Assuming your DataFrames are df1 and df2 and the article IDs are in a column named 'Article ID'

# Extract article IDs from both DataFrames
article_ids_df1 = finaltrain['Article ID'].unique()
article_ids_df2 = finaltest['Article ID'].unique()

# Find common article IDs
common_ids_in_df1 = finaltrain[finaltrain['Article ID'].isin(article_ids_df2)]['Article ID'].unique()
common_ids_in_df2 = finaltest[finaltest['Article ID'].isin(article_ids_df1)]['Article ID'].unique()

# Check if there are any common article IDs
if len(common_ids_in_df1) > 0:
    print("Common article IDs:", common_ids_in_df1)
else:
    print("There are no common article IDs in the two DataFrames.")


There are no common article IDs in the two DataFrames.


In [ ]:
len(common_ids_in_df1)

0